# Практика
Используемые библиотеки

In [1]:
import random

import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
df


,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [ ]:
# Ваше решение
df['Норматив_list'] = df.Норматив.apply(lambda x: re.split('-', (re.sub(',', '.', re.sub('[А-Яа-я ]', '', x)))))

df['Норматив_min'] = df.Норматив_list.apply(lambda x: float(x[0]) if len(x) > 1 else float(0))
df['Норматив_max'] = df.Норматив_list.apply(lambda x: float(x[1]) if len(x) > 1 else float(x[0]))

df['Результат анализа value'] = df['Результат анализа'].apply(lambda x: 0 if len(re.sub('[А-я/]', '', x)) == 0 else float(re.sub('[А-я/]', '', x)))
df['result'] = (df['Результат анализа value'] >= df['Норматив_min']) & (df['Результат анализа value'] <= df['Норматив_max'])
df.result = df.result.apply(lambda x: 'Good' if x else 'Bad')
df = (df.drop(['Норматив_min', 'Норматив_max', 'Норматив_list'], axis=1)).set_index('Показатель')
df

,Единица измерений,Результат анализа,Норматив,Результат анализа value,result
Показатель,,,,,
pH,единицы pH,8.4,в пределах 6-9,8.400,Good
Запах,баллы,1,не более 2-3,1.000,Bad
Цветность,градусы,б/цвета,не более 30,0.000,Good
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,9.200,Good
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",0.420,Good
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.017,Good
Нитраты (по NO3),мг/дм3,24,не более 45,24.000,Good
Фосфаты (P),мг/дм3,0.36,"не более 3,5",0.360,Good
Хлориды (Cl),мг/дм3,200,не более 350,200.000,Good


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
# Ваше решение
tryes = int(10e6)
fruits = [[['апельсин']*5 + ['яблоко']*4]]*int(tryes)
df = pd.DataFrame(fruits, columns=['basket'])
df['attempts'] = df.basket.apply(lambda x: 1 if set(np.random.choice(x, 3)) == set(['апельсин']) else 0)
print(df.attempts.sum()/tryes)

In [ ]:
fruits_other = ['апельсин', 'яблоко']
df_test = pd.DataFrame([[fruits_other]]*tryes, columns=['fruits'])
df_test['fruits'] = df_test.fruits.apply(lambda x: 1 if set(np.random.choice(x, 3, replace=True, p=[5/9, 4/9])) == set(['апельсин']) else 0)
print(df_test.fruits.sum()/tryes)

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение
details = ['стандартная'] * 7 + ['нестандартная'] * 3
df = pd.DataFrame([[details]]*tryes, columns=['all_details'])
df.all_details = df.all_details.apply(lambda x: (np.random.permutation(x)))
df['try1'] = df.all_details.apply(lambda x: [x[0], np.random.permutation(x[1:])[0]])
len(df[df['try1'] == pd.Series([['стандартная', 'нестандартная']]*tryes)])/tryes

In [ ]:
details_new = ['стандартная', 'нестандартная']
df_details = pd.DataFrame([[details_new]]*tryes, columns=['details'])
df_details['details'] = df_details['details'].apply(lambda x: 1 if list(np.random.choice(x, 2, replace=True, p=[7/10, 3/10])) == details_new else 0)
print(df_details['details'].sum()/tryes)